In [11]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

import os


df_train = pd.read_csv('../data/interim/train_data/2016_train.csv')
data_dict = pd.read_json('../data/interim/data_dictionary_v1.json')
df_test = pd.read_csv('../data/interim/test_data/2016_test.csv')

Show data_dict with column descriptions:

In [2]:
print(f'Train shape = {df_train.shape} , test shape = {df_test.shape}')

Train shape = (5521, 87) , test shape = (2928, 53)


A crapload of features, let's do a quick describe:

In [3]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
final.output.concentrate_fe,5460.0,5.187098,2.291511,0.000000e+00,4.261007,5.190401,6.297080,1.600195e+01
final.output.concentrate_pb,5460.0,9.095405,3.198236,0.000000e+00,8.743032,9.964454,10.894291,1.703190e+01
final.output.concentrate_sol,5162.0,8.346959,3.989839,0.000000e+00,6.813081,8.748396,10.848410,1.812485e+01
final.output.concentrate_zn,5461.0,39.154972,12.951382,0.000000e+00,40.434038,43.611745,45.400805,5.157188e+01
final.output.recovery,4949.0,64.347394,10.381570,0.000000e+00,60.749253,65.792101,70.159212,1.000000e+02
final.output.tail_fe,5461.0,8.779861,3.323955,0.000000e+00,7.498053,9.540737,11.013164,1.492167e+01
final.output.tail_pb,5454.0,1.882659,0.878259,0.000000e+00,1.418979,1.957857,2.438280,4.626952e+00
final.output.tail_sol,5445.0,10.865604,4.818499,0.000000e+00,9.175401,11.269699,13.391020,2.231773e+01
final.output.tail_zn,5461.0,2.836827,1.170430,0.000000e+00,2.427488,2.906540,3.483971,8.197408e+00
primary_cleaner.input.copper_sulfate,5179.0,121.339683,45.466436,1.002061e-03,95.612438,121.766753,148.112158,2.519999e+02


There are some columns with `output` in the name.. Those are not available at the test prediction, and thus not super useful yet. Although in TODO list I'd add a possibility to build the model using `output` predictors and compare on local CV the quality of predictions with the model that uses test columns only

In [5]:
df_train.filter(regex = 'output',axis =1).head()

,final.output.concentrate_fe,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_zn,final.output.recovery,final.output.tail_fe,final.output.tail_pb,final.output.tail_sol,final.output.tail_zn,primary_cleaner.output.concentrate_fe,...,rougher.output.concentrate_zn,rougher.output.recovery,rougher.output.tail_fe,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_zn,secondary_cleaner.output.tail_fe,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_zn
0,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,8.547551,...,19.793808,87.107763,5.008018,0.508728,19.154297,1.170244,14.500184,4.694824,8.764648,2.606185
1,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,8.558743,...,20.050975,86.843261,4.955378,0.536689,18.965234,1.184827,14.265474,4.592547,9.001494,2.488248
2,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,8.603505,...,19.737170,86.842308,4.843460,0.546416,18.808533,1.162562,14.115662,4.624775,8.842896,2.458187
3,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,7.221879,...,19.320810,87.226430,4.655288,0.542525,19.330200,1.079755,13.732071,4.482453,9.122857,2.322062
4,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,9.089428,...,19.216101,86.688794,4.552800,0.515438,19.267422,1.012642,14.079996,4.470737,8.871028,2.330448


Lets quickly run `pandas-profiling` report on training data to see the correlation / high cardinality / low variance variables

In [ ]:
import pandas_profiling as pdpr
rep_train = pdpr.ProfileReport(df_train)
rep_test = pdpr.ProfileReport(df_test)

rep_train.to_file(outputfile="../reports/train_output.html")
rep_test.to_file(outputfile="../reports/test_output.html")


Seems like there are a lot of features correlated, mainly corresponding to the same main *source* of information (e.g. `feed` , `floatbank` features). We should not necessarily throw one of them away, but rather eliminate them in a justifiable fashion (RFE?)

Let's do a few more plots:

- each feature vs time
- pair-wise plots between features

and dump them into a `png` file

First, the pairplot: (only for columns in the `df_test`). I wi ll also use R's ggpairs, since they are much faster than python's seaborn

In [32]:
def corrfunc(x, y, **kws):
    r, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate(u"\u03C1 = {:.2f}".format(r), #unicode code for lowercase rho (ρ)
                xy=(.1, .9), xycoords=ax.transAxes)
cols =df_test.columns.difference(['date'])
#print(cols)
df_train_small = df_train[cols]
df_test_small = df_test[cols]
#g = sns.pairplot(df_train[cols])
#g.map_lower(corrfunc)
#g.get_figure().savefig('../reports/figures/train_pairs.png',figsize  = (40,40))
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R -i df_train_small -i df_test_small
library(GGally)
library(ggplot2)
p = ggpairs(df_train_small)
ggsave('pairs-train.png',p,width = 40,height =40)



In [ ]:
%%R -i df_train_small -i df_test_small
library(GGally)
library(ggplot2)
ptest = ggpairs(df_test_small)

ggsave('pairs-test.png',ptest,width = 40,height =40)

In [ ]:
g_test = sns.pairplot(df_test[cols])
g_test.map_lower(corrfunc)
g_test.get_figure().savefig('../reports/figures/train_pairs.png',figsize  = (40,40))

In [ ]:
# Save the figures somewhere

